In [33]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import math

In [19]:
file_path = './dataset/'
file_name_A = 'cityA_groundtruthdata.csv.gz'
file_name_B = 'cityB_challengedata.csv.gz'
file_name_C = 'cityC_challengedata.csv.gz'
file_name_D = 'cityD_challengedata.csv.gz'

# CityA

In [74]:
traj_A = pd.read_csv(file_path + file_name_A, compression='gzip')
traj_A[:10]

,uid,d,t,x,y
0,0,0,1,79,86
1,0,0,2,79,86
2,0,0,8,77,86
3,0,0,9,77,86
4,0,0,19,81,89
5,0,0,20,82,88
6,0,0,21,81,89
7,0,0,22,81,89
8,0,0,24,76,86
9,0,0,25,75,82


In [81]:
print(type(traj_A))
traj_A_np = traj_A.to_numpy()
print(type(traj_A_np))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


## Distance Statics

In [84]:
def Euclidean_Distance(point1, point2):
    """
    计算二维空间中两个点之间的欧式距离。
    
    参数:
    point1 (tuple): 第一个点的坐标，格式为 (x1, y1);
    point2 (tuple): 第二个点的坐标，格式为 (x2, y2)
    
    返回值:
    float: 两个点之间的欧式距离
    """
    x1, y1 = point1
    x2, y2 = point2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

### distance/peruser

In [130]:
dis_peruser = []
uid, dis = -1, 0

for row in traj_A_np:
    if uid != row[0]:
        if uid != -1:
            dis_peruser.append([uid, dis])
        uid = row[0]
        x, y = row[3], row[4]
        dis = 0
        continue
    dis += Euclidean_Distance((x,y),(row[3],row[4]))
    x, y = row[3], row[4]
dis_peruser.append([uid, dis])


In [145]:
dis_peruser = np.array(dis_peruser)
dis_peruser.shape, dis_peruser[1], dis_peruser[-1]

((100000, 2),
 array([1.00000000e+00, 9.66822613e+03]),
 array([99999.        ,  4239.04533656]))

In [143]:
np.mean(dis_peruser[:, 1])

6386.78283373992

### distance/(perday*peruser)

In [150]:
dis_perday_peruser = []
uid, day, dis = -1, -1, 0

for row in traj_A_np:
    if (uid != row[0]) or (day != row[1]):
        if (uid != -1) and (day != -1):
            dis_perday_peruser.append([uid, day, dis])
        uid = row[0]
        day = row[1]
        x, y = row[3], row[4]
        dis = 0
        continue
    dis += Euclidean_Distance((x,y),(row[3],row[4]))
    x, y = row[3], row[4]
dis_perday_peruser.append([uid, day, dis])

In [151]:
dis_perday_peruser = np.array(dis_perday_peruser)
dis_perday_peruser.shape, dis_perday_peruser[0], dis_perday_peruser[-1]

((7147619, 3),
 array([ 0.        ,  0.        , 68.90799852]),
 array([9.99990000e+04, 7.40000000e+01, 2.57488732e+02]))

In [152]:
np.mean(dis_perday_peruser[:, 2])

84.31122178253938

# Spactial Statics

# CityB